In [ ]:
## ensemble

In [18]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-30 15:58:17 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,778467,41.6,1454587,77.7,1454587,77.7
Vcells,1628338,12.5,1573177630,12002.4,1704760672,13006.3


In [19]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset1 <- fread("~/datasets/competencia_02_crudo.csv.gz" )
dataset2<- fread("~/datasets/competencia_03_crudo.csv.gz" )

dataset <- rbind(dataset1, dataset2)


# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-30 15:58:18 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,779266,41.7,1454587,77.7,1454587,77.7
Vcells,1301424852,9929.1,1887893156,14403.5,1704760672,13006.3


[1] "2025-11-30 15:58:49 UTC"

In [20]:
# 1) Traer dataset del mes y ganancia real
dfuture <- dataset[foto_mes == 202107]
dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]


# 2) Traer archivo de predicciones en test
files <- list.files("/home/guadalesandro/buckets/b1/exp",
                    pattern="^prediccion_mid\\.txt$",
                    recursive=TRUE,
                    full.names=TRUE)


# 3) leer todos los archivos con header=TRUE
pred_list <- lapply(files, function(f){
  fread(f, sep="\t", header=TRUE, encoding="UTF-8")
})

pred <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)


# 4) Une la predicción y la ganancia real por número de cliente

pred[, numero_de_cliente := as.character(numero_de_cliente)]
dfuture[, numero_de_cliente := as.character(numero_de_cliente)]
pred[, prob := as.numeric(prob)]


pred <- merge(
  pred[, .(numero_de_cliente, prob)],
  dfuture[, .(numero_de_cliente, ganancia)],
  by = "numero_de_cliente",
  all.x = TRUE
)

# 4) Promedia la probabilidad por cliente
pred_ensamble <- pred[, .(
  prob_mean = mean(prob),
  ganancia = first(ganancia)
), by = numero_de_cliente]

# 5) Ordenar y ganancia acumulada
setorder(pred_ensamble, -prob_mean)
pred_ensamble[, gan_acum := cumsum(ganancia)]

# 6) Ganancia final de los 11000 más probables
envios <- 11500
ganancia_final <- pred_ensamble[envios, gan_acum]
ganancia_final

print(files)

[1] 426800000

 [1] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_b2/prediccion_mid.txt"
 [2] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_d/prediccion_mid.txt" 
 [3] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e/prediccion_mid.txt" 
 [4] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e2/prediccion_mid.txt"
 [5] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_f/prediccion_mid.txt" 
 [6] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_g/prediccion_mid.txt" 
 [7] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_h/prediccion_mid.txt" 
 [8] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_j/prediccion_mid.txt" 
 [9] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k/prediccion_mid.txt" 
[10] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k2/prediccion_mid.txt"
[11] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k3/prediccion_mid.txt"
[12] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k4/prediccion_mid.txt"
[13] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_l/prediccion_mid.txt" 
[14] "/home/guadalesandro

In [ ]:
## predicción final ensamble

In [21]:
dfuture <- dataset[foto_mes == 202109]

In [28]:
# 1 - Levanta predicciones y las concatena
files <- list.files("/home/guadalesandro/buckets/b1/exp", #/C3
                    pattern="^prediccion\\.txt$",
                    recursive=TRUE,
                    full.names=TRUE)

print(files)

pred_list <- lapply(files, function(f){
  fread(f, sep="\t", header=TRUE, encoding="UTF-8")})
pred <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)


pred[, numero_de_cliente := as.character(numero_de_cliente)]
pred[, prob := as.numeric(prob)]

#promedia probabilidades y las ordena
pred_ensamble <- pred[, .(prob_mean = mean(prob, na.rm=TRUE)), by = numero_de_cliente]
setorder(pred_ensamble, -prob_mean)

# Filtrar solo clientes de dfuture
clientes_future <- unique(dfuture$numero_de_cliente)
pred_ensamble <- pred_ensamble[numero_de_cliente %in% clientes_future]


envios <- 11500
pred_ensamble[, Predicted := 0L]
pred_ensamble[1:envios, Predicted := 1L]

cat("Cantidad de registros en el archivo final:", nrow(pred_ensamble), "\n")

#Guarda el archivo en carpeta final
output_dir <- "/home/guadalesandro/dmeyf2025/babycode/R_C3/C3_final"

# crear la carpeta si no existe
if(!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)
archivo_final <- file.path(output_dir, "C3_final_Alesandro.csv")

fwrite(
  pred_ensamble[, .(numero_de_cliente, Predicted)],
  file = archivo_final,
  sep = ","
)

cat("Archivo guardado en:", archivo_final, "\n")


 [1] "/home/guadalesandro/buckets/b1/exp/apo-505_a/prediccion.txt"                                                 
 [2] "/home/guadalesandro/buckets/b1/exp/apo-505_a2/prediccion.txt"                                                
 [3] "/home/guadalesandro/buckets/b1/exp/apo-505_base_integrada/prediccion.txt"                                    
 [4] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_b/prediccion.txt"                                              
 [5] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_c/prediccion.txt"                                              
 [6] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_d/prediccion.txt"                                              
 [7] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e/prediccion.txt"                                              
 [8] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e2/prediccion.txt"                                             
 [9] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_f/prediccion.txt"   